
<font size="5">**On the comparison of different convexified power flow models in radial network**</font> 

***
This repository contains the supplementary material for the paper entitled "On the comparison of different convexified power flow models in radial network" published on IEEE Xplore: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9814764

In the repsitory, the code of following sections are provided:
1. Read data from MATPOWER
2. Bus injection model
3. Branch flows model

# Read data from MATPOWER

In [1]:
#Read data
import cvxpy as cvx
import numpy as np
import math
import mosek
import gurobipy
# to run the following commands, please follow  https://www.instructables.com/id/Call-MATLAB-Script-and-Function-From-Python/
import matlab.engine
eng = matlab.engine.start_matlab()

##Radial network test cases
#case = "case141"
case ="case33bw"
#case = "case18"

ret = eng.Read_RadialNet_Matpower(case)  #case33bw, case123  # these command is equivalent to run Read_Matpower_Radial(case33bw) in Matlab
import scipy.io as spio #Reading Matlab mat data as Python dictionary
data = spio.loadmat('Radial_Netdata', squeeze_me=True) #taken from Matlab command Read_Matpower_Radial(Matpower_case_format);
#data contains the following parameters
#'Sbase', 'Pgmax', 'Pgmin', 'Qgmax', 'Qgmin', 'GenVarCost0', 'GenVarCost', 'G', 'B', 'MVAlim', 'connex', 'Pd', 'Qd', 'Vmax', 'Vmin','Gen2Bus'
r = data['r']
x = data['x']
slack_bus =0
#slack_bus =55 #case 123 slackbus is 56 (Python index count from 0)
#slack_bus =113
I = len(data['Pd'])
G = data['G']
B = -data['B']
MVAlim =data['MVAlim']
status = data['status']

Define a connex function to check whether a pair of node $i,j=\ell \in \mathcal{L}$

In [2]:
def connex(i,j,data): #return 1 of (i,j) or (j,i) is a "connected" line
    temp = False
    for line in range(data['L']):
        z1= (i== data['from_bus'][line]-1) and (j== data['to_bus'][line]-1)
        z2= (j== data['from_bus'][line]-1) and (i== data['to_bus'][line]-1)
        z3= (data['status'][line]==1) #some lines are open switches
        z= (z1 or z2) and z3
        temp = temp or z
    return temp

Normalize the data from MATPOWER

In [3]:
print(MVAlim)
if case =="case18":
    scale = 10
else:
    scale = 1
data['Sbase']= scale*data['Sbase']
r = r*scale
x = x*scale
data['gs'] = data['gs']/scale
data['bs'] = data['bs']/scale

[270 270 270 270 270 270 270 270 270 270 270 270 270 270 270 270 270 270
 270 270 270 270 270 270 270 270 270 270 270 270 270 270]


# Bus injection model

Objective function: $\min P^{\sf grid}$

Subject to:

> $(\underline{V}_i)^2 \leq V_{i}^{sq} \leq (\overline{V}_i)^2 , \quad \forall i \in \mathcal{I}$
    
> $P_{ij} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( r_{\ell} V_{i}^{sq} - r_\ell R_{\ell} + x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

> $Q_{ij} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( x_{\ell} V_{i}^{sq} - x_\ell R_{\ell} - x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

> $P_{ji} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( r_{\ell} V_{j}^{sq} - r_\ell R_{\ell} - x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

> $Q_{ji} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( x_{\ell} V_{j}^{sq} - x_\ell R_{\ell} + x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

> $I^{sq}_{\ell} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( V_{i}^{sq} + V_{j}^{sq} - 2R_{\ell} \right) \geq 0, \quad  \forall \ell = ij \in \mathcal{L}$

> $P^d_{i} + \sum_{j \in \mathcal{N} \left(i \right)} P_{ij} = 0, \quad \forall i \in \mathcal{I}$

> $Q^d_{i} + \sum_{j \in \mathcal{N} \left( i \right)} Q_{ij} = 0, \quad \forall i \in \mathcal{I}$

> $P^{\sf grid} = \sum\limits_{j \in \mathcal{N}(1)} P_{1j}$
> $Q^{\sf grid} = \sum\limits_{j \in \mathcal{N}(1)} Q_{1j}$

> $(R_{\ell})^2 + (T_{\ell})^2 \leq V_{i}^{sq} V_{j}^{sq}, ~ R_{\ell}  \geq 0,  \quad \forall \ell = ij \in \mathcal{L}$

> $I^{sq}_{\ell} \geq 0,  \quad \forall \ell \in \mathcal{L}$
***

Let's define variables: $P_{ij}, Q_{ij}, R_\ell, T_\ell, V^{sq}_i, I^{sq}_i$,
and define the constraint set

In [4]:
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
R = cvx.Variable(data['L'],nonneg=True) 
T = cvx.Variable(data['L'])            
Vsq  = cvx.Variable(I) 
Isq  = cvx.Variable(data['L'],nonneg=True)

const = []

## Voltage limits constraint:
$(\underline{V}_i)^2 \leq V_{i}^{sq} \leq (\overline{V}_i)^2 , \quad \forall i \in \mathcal{I}$

In [5]:
const += [Vsq[i] >= (data['Vmin'][i])**2 for i in range(I) if i != slack_bus]
const += [Vsq[i] <= (data['Vmax'][i])**2 for i in range(I) if i != slack_bus]

if case == "case18":
    const += [Vsq[slack_bus] == 1.1]
else:
    const += [Vsq[slack_bus] == 1]

## The line power flow equations $P_{ij}, P_{ji}, Q_{ij}, Q_{ji}$ and the squared line current $I^{sq}_\ell$:  
$P_{ij} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( r_{\ell} V_{i}^{sq} - r_\ell R_{\ell} + x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$Q_{ij} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( x_{\ell} V_{i}^{sq} - x_\ell R_{\ell} - x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$P_{ji} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( r_{\ell} V_{j}^{sq} - r_\ell R_{\ell} - x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$Q_{ji} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( x_{\ell} V_{j}^{sq} - x_\ell R_{\ell} + x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

In [6]:
for line in range(data['L']):
    i = data['from_bus'][line]-1
    j = data['to_bus'][line]-1       
    if status[line] ==1:
        const += [P[i,j] == (r[line]*Vsq[i] - r[line]*R[line]+ x[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [Q[i,j] == (x[line]*Vsq[i] - x[line]*R[line]- r[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [P[j,i] == (r[line]*Vsq[j] - r[line]*R[line]- x[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [Q[j,i] == (x[line]*Vsq[j] - x[line]*R[line]+ r[line]*T[line])/(r[line]**2 + x[line]**2)]        
        const += [Isq[line]  == (Vsq[i]+Vsq[j] - 2*R[line])/(r[line]**2 + x[line]**2)]

## The active and reactive power balance constraints:  
$P^d_{i} + \sum_{j \in \mathcal{N} \left(i \right)} P_{ij} = 0, \quad \forall i \in \mathcal{I}$

$Q^d_{i} + \sum_{j \in \mathcal{N} \left( i \right)} Q_{ij} = 0, \quad \forall i \in \mathcal{I}$

In [7]:
bus_Pinjection = [-sum(P[i,j] for j in range(I) if connex(i,j,data)) for i in range(I)]
bus_Qinjection = [-sum(Q[i,j] for j in range(I) if connex(i,j,data)) for i in range(I)]

const += [(bus_Pinjection[i]+data['gs'][i]*Vsq[i])== data['Pd'][i]/data['Sbase']  for i in range(I) if i != slack_bus]
const += [(bus_Qinjection[i]+data['bs'][i]*Vsq[i]) == data['Qd'][i]/data['Sbase'] for i in range(I) if i != slack_bus]

## The SOC constraint:  
The rotated conic constraint $(R_{\ell})^2 + (T_{\ell})^2 \leq V_{i}^{sq} V_{j}^{sq}, ~ R_{\ell}  \geq 0,  \quad \forall \ell = ij \in \mathcal{L}$ 
can be casted into 

$\sqrt{R^2_{\ell} + T^2_{\ell} + \left( \frac{V_{i}^{sq} - V_{j}^{sq}}{2} \right)^2} \leq \frac{V_{i}^{sq} + V_{j}^{sq}}{2}, \forall \ell = ij \in \mathcal{L}$

There are two methods to implement the SOC constraint in CVXPY:
- ***quad_over_lin*** function
- ***norm2*** function

In [8]:
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
#        const += [cvx.quad_over_lin(cvx.bmat([[R[line], T[line]]]), Vsq[i]) <= Vsq[j]]
        const += [cvx.norm2(cvx.bmat([[R[line], T[line], 0.5*(Vsq[i] - Vsq[j])]])) <= 0.5*(Vsq[i]+ Vsq[j])]

## Objective function: $\min P^{\sf grid}$

In this paper, we tested the model with CPLEX and MOSEK. Many other solvers can be called by CVXPY if installed. For detail of supported solvers by CVXPY, refer to: https://www.cvxpy.org/tutorial/advanced/index.html

In [9]:
Pgrid = -bus_Pinjection[slack_bus]*data['Sbase']
Qgrid = -bus_Qinjection[slack_bus]*data['Sbase']

prob = cvx.Problem(cvx.Minimize(Pgrid), const) 

#Define a desired solver at user's preference:
prob.solve(solver='MOSEK', verbose=True, warm_start = True)
#prob.solve(solver='CPLEX', verbose=True, warm_start = True) 

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) Jul 20 10:50:51 AM: Your problem has 2307 variables, 321 constraints, and 0 parameters.
(CVXPY) Jul 20 10:50:51 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 20 10:50:51 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 20 10:50:51 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 20 10:50:51 AM: Compiling problem (target solver=MOSEK).
(CVXPY) Jul 20 10:50:51 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

3.917265542651016

## Export the optimal solution

In [10]:
print('Active power loss:', Pgrid.value-sum(data['Pd']), 'MW')

print("BUS DATA")
print('Imported active power from main grid:', Pgrid.value,'MW')
print('Imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus \t Voltage")
for i in range(I):
    print(i+1,'\t', round(math.sqrt(Vsq[i].value), 4))
print('================================================================================')
print("LINE DATA")
print("Bus \t Current")
for line in range(data['L']):   
    if status[line] ==1:
        print(i+1,'\t',round(math.sqrt(Isq[line].value), 4))
print('=================================END CVXPY======================================')
print('================================================================================')

Active power loss: 0.20226554265101493 MW
BUS DATA
Imported active power from main grid: 3.917265542651016 MW
Imported reactive power from main grid: 2.434882825486717 MVAr
Bus 	 Voltage
1 	 1.0
2 	 0.997
3 	 0.9829
4 	 0.9755
5 	 0.9681
6 	 0.9497
7 	 0.9462
8 	 0.9413
9 	 0.9351
10 	 0.9293
11 	 0.9284
12 	 0.9269
13 	 0.9208
14 	 0.9185
15 	 0.9171
16 	 0.9157
17 	 0.9137
18 	 0.9131
19 	 0.9965
20 	 0.9929
21 	 0.9922
22 	 0.9916
23 	 0.9794
24 	 0.9727
25 	 0.9694
26 	 0.9477
27 	 0.9452
28 	 0.9337
29 	 0.9255
30 	 0.922
31 	 0.9178
32 	 0.9169
33 	 0.9166
LINE DATA
Bus 	 Current
33 	 0.046
33 	 0.041
33 	 0.0295
33 	 0.028
33 	 0.0274
33 	 0.0128
33 	 0.0104
33 	 0.0081
33 	 0.0074
33 	 0.0065
33 	 0.006
33 	 0.0054
33 	 0.0046
33 	 0.0031
33 	 0.0024
33 	 0.0018
33 	 0.001
33 	 0.0035
33 	 0.003
33 	 0.0019
33 	 0.001
33 	 0.0106
33 	 0.0096
33 	 0.0048
33 	 0.0142
33 	 0.0136
33 	 0.0131
33 	 0.0125
33 	 0.0111
33 	 0.0051
33 	 0.0032
33 	 0.0004
==============================

# Branch flows model

Objective function: $\min P^{\sf grid}$

Subject to:

> $(\underline{V}_{i})^2 \leq V_i^{sq} \leq (\overline{V}_{i})^2, \quad \forall i \in \mathcal{I}$

> $\sum_{k: k \to i}\big(P_{ki} - r_{ki} I^{sq}_{ki} \big) - \sum_{j: i \to j } P_{ij} = P^{d}_{i}, ~~ \forall i \in \mathcal{I}$

> $\sum_{k: k \to i}\big(Q_{ki} - x_{ki} I^{sq}_{ki} \big) - \sum_{j: i \to j } Q_{ij} = Q^{d}_{i}, ~~ \forall j \in \mathcal{I}$

> $P^{\sf grid} = \sum\limits_{j: 1 \to j} P_{1j},~~ Q^{\sf grid} = \sum\limits_{j: 1\to j } Q_{1j}$

> $V_j^{sq} = V_i^{sq} - 2(r_{\ell} P_{ij} + x_{\ell} Q_{ij})+(r_{\ell}^2 + x_{\ell}^2)I^{sq}_{\ell}, \quad \forall ij=\ell \in \mathcal{L}$

> $I^{sq}_{\ell} \geq 0,  \quad \forall \ell \in \mathcal{L}$

> $P_{ij}^2 + Q_{ij}^2 \leq V_i^{sq} I^{sq}_{\ell}, ~~ \forall\ell=ij\in \mathcal{L}$
***

Let's define variables: $P_{ij}, Q_{ij}, V^{sq}_i, I^{sq}_i$, and the constraint set

In [11]:
#data = spio.loadmat('Radial_Netdata', squeeze_me=True)
#r = data['r']
#x = data['x']
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
Vsq  = cvx.Variable(I)
Isq  = cvx.Variable(shape=(data['L'],), nonneg=True)

const = []

## Voltage limits constraint:
$(\underline{V}_i)^2 \leq V_{i}^{sq} \leq (\overline{V}_i)^2 , \quad \forall i \in \mathcal{I}$

In [12]:
const += [Vsq[i] >= (data['Vmin'][i])**2 for i in range(I) if i!=slack_bus]
const += [Vsq[i] <= (data['Vmax'][i])**2 for i in range(I) if i!=slack_bus]
if case == "case18":
    const += [Vsq[slack_bus] == 1.1]
else:
    const += [Vsq[slack_bus] == 1]

## Acitve and reacive power balance constraints:
$\sum_{k: k \to i}\big(P_{ki} - r_{ki} I^{sq}_{ki} \big) - \sum_{j: i \to j } P_{ij} = P^{d}_{i}, ~~ \forall i \in \mathcal{I}$

$\sum_{k: k \to i}\big(Q_{ki} - x_{ki} I^{sq}_{ki} \big) - \sum_{j: i \to j } Q_{ij} = Q^{d}_{i}, ~~ \forall j \in \mathcal{I}$

In [13]:
P_outflow_of = [sum(P[j, data['to_bus'][line]-1] for line in range(data['L']) 
                  if  ((status[line]==1) and (j==data['from_bus'][line]-1)))  for j in range(I)]     

Q_outflow_of = [sum(Q[j, data['to_bus'][line]-1] for line in range(data['L']) 
                  if  ((status[line]==1) and (j==data['from_bus'][line]-1)) ) for j in range(I)] 

P_inflow_of  = [sum(P[data['from_bus'][line]-1,j] - r[line]*Isq[line] 
                        for line in range(data['L']) if  ((status[line]==1) and (j==data['to_bus'][line]-1)) ) 
                        for j in range(I)]  

Q_inflow_of  = [sum(Q[data['from_bus'][line]-1,j] - x[line]*Isq[line] 
                        for line in range(data['L'])  if  ((status[line]==1) and (j==data['to_bus'][line]-1)))  
                        for j in range(I)] 
const += [data['gs'][j]*Vsq[j] - (P_outflow_of[j] - P_inflow_of[j]) == data['Pd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]
const += [data['bs'][j]*Vsq[j] - (Q_outflow_of[j] - Q_inflow_of[j]) == data['Qd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]

## Voltage drop equation:
$V_j^{sq} = V_i^{sq} - 2(r_{\ell} P_{ij} + x_{\ell} Q_{ij})+(r_{\ell}^2 + x_{\ell}^2)I^{sq}_{\ell}, \quad \forall ij=\ell \in \mathcal{L}$

In [14]:
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        const +=[ Vsq[j] == Vsq[i] - 2*(r[line]*P[i,j]+x[line]*Q[i,j]) + (r[line]**2 + x[line]**2)*Isq[line]]

## The SOC constraint:
The rotated conic constraint $P_{ij}^2 + Q_{ij}^2 \leq V_i^{sq} I^{sq}_{\ell}, ~~ \forall\ell=ij\in \mathcal{L}$ 
can be casted into 

$\sqrt{P^2_{ij}+Q^2_{ij} + \Bigg(\frac{V_i^{sq} - I_{\ell}^{sq}}{2} \Bigg)^2} \leq \frac{V_i^{sq} + I_{\ell}^{sq}}{2}, \forall\ell=ij\in \mathcal{L}$

There are two methods to implement the SOC constraint in CVXPY:
- ***quad_over_lin*** function
- ***norm2*** function

In [15]:
for line in  range(data['L']): 
    if status[line]==1:
        const += [cvx.norm2(cvx.bmat( [[P[data['from_bus'][line]-1,data['to_bus'][line]-1], 
                                                Q[data['from_bus'][line]-1,data['to_bus'][line]-1],
                                               0.5*(Vsq[data['from_bus'][line]-1]-Isq[line])]]))
                                <= 0.5*(Vsq[data['from_bus'][line]-1]+Isq[line])]

## Objective function: $\min P^{\sf grid}$

In this paper, we tested the model with CPLEX and MOSEK. Many other solvers can be called by CVXPY if installed. For detail of supported solvers by CVXPY, refer to: https://www.cvxpy.org/tutorial/advanced/index.html

In [16]:
Pgrid = P_outflow_of[slack_bus]*data['Sbase']
Qgrid = Q_outflow_of[slack_bus]*data['Sbase']  
  
prob = cvx.Problem(cvx.Minimize(Pgrid), const)

#prob = Problem(Minimize(obj*data['Sbase']), const) 
prob.solve(solver='CPLEX', verbose=True, warm_start = True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) Jul 20 10:51:19 AM: Your problem has 2243 variables, 193 constraints, and 0 parameters.
(CVXPY) Jul 20 10:51:19 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 20 10:51:19 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 20 10:51:19 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 20 10:51:19 AM: Compiling problem (target solver=CPLEX).
(CVXPY) Jul 20 10:51:19 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

3.9176763660355745

## Export the optimal solution

In [17]:
print('imported grid:', Pgrid.value, 'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')

print('loss:', Pgrid.value-sum(data['Pd']))

print('voltage:', [math.sqrt(Vsq[i].value) for i in range(I)])

print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(Vsq[i].value), 5))

imported grid: 3.9176763660355745 MW
imported reactive power from main grid: 2.4351406426080486 MVAr
loss: 0.20267636603557326
voltage: [1.0, 0.9970322604631742, 0.9829379869239869, 0.9754564187861315, 0.9680592398283269, 0.9496581892367615, 0.9461726258011464, 0.9413284509114908, 0.9350593875142781, 0.9292444400314803, 0.9283844349003093, 0.9268848546123404, 0.9207717649866609, 0.9185050101162173, 0.917092698182686, 0.9157247788018156, 0.9136975657669528, 0.9130904990171635, 0.9965038965580432, 0.9929263014859355, 0.9922217979279909, 0.9915843789969369, 0.979352261188058, 0.9726811052979137, 0.9693561168526644, 0.9477289221709727, 0.9451651766739038, 0.933725594693745, 0.9255074924785229, 0.9219500721166075, 0.9177889013848247, 0.9168734802989371, 0.9165898367128407]
 BUS DATA
imported active power from main grid: 3.9176763660355745 MW
imported reactive power from main grid: 2.4351406426080486 MVAr
Bus    Voltage
1    1.0
2    0.99703
3    0.98294
4    0.97546
5    0.96806
6    0.9496